In [1]:
import yfinance as yf
import numpy as np

from sklearn.covariance import LedoitWolf

In [11]:
df = yf.download('GC=F NVDA IONQ', start='2021-01-04')

[*********************100%***********************]  3 of 3 completed


In [17]:
cov_matrix = df['Close'].cov().values
cov_matrix

array([[6.39268102e+04, 9.08143245e+01, 8.78253100e+03],
       [9.08143245e+01, 1.89140963e+01, 2.23801733e+01],
       [8.78253100e+03, 2.23801733e+01, 1.35327885e+03]])

In [18]:
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)

In [19]:
eigenvalues

array([1.80213230e+01, 1.44734549e+02, 6.51362473e+04])

In [20]:
eigenvectors

array([[-0.00915256, -0.13610739, -0.99065181],
       [-0.99699597,  0.07744022, -0.00142847],
       [ 0.07691072,  0.98766279, -0.1364073 ]])

In [21]:
# Shrink eigenvalues (basic example)
mean_eigenvalue = np.mean(eigenvalues)
shrunk_eigenvalues = np.maximum(eigenvalues, mean_eigenvalue)

In [22]:
mean_eigenvalue

21766.33438051346

In [23]:
shrunk_eigenvalues

array([21766.33438051, 21766.33438051, 65136.24726926])

In [24]:
# Reconstruct the covariance matrix with shrunk eigenvalues
rie_cov_matrix = eigenvectors @ np.diag(shrunk_eigenvalues) @ eigenvectors.T

In [25]:
rie_cov_matrix

array([[6.43291770e+04, 6.13734990e+01, 5.86066888e+03],
       [6.13734990e+01, 2.17664229e+04, 8.45079262e+00],
       [5.86066888e+03, 8.45079262e+00, 2.25733162e+04]])

In [26]:
data = df['Close'].values

In [27]:
# Compare with Ledoit-Wolf shrinkage
lw = LedoitWolf()
lw_cov = lw.fit(data).covariance_

In [28]:
lw_cov

array([[6.36531042e+04, 9.02727954e+01, 8.73016045e+03],
       [9.02727954e+01, 1.26293187e+02, 2.22467195e+01],
       [8.73016045e+03, 2.22467195e+01, 1.45270107e+03]])

In [29]:
def compute_rie(cov_matrix):
    """
    Computes the Rotation-Invariant Estimator (RIE) for a covariance matrix.

    Parameters:
    - cov_matrix (np.ndarray): The input covariance matrix (p x p).

    Returns:
    - np.ndarray: The improved RIE covariance matrix.
    """
    # Eigen decomposition
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    
    # Shrink eigenvalues (basic example)
    mean_eigenvalue = np.mean(eigenvalues)
    shrunk_eigenvalues = np.maximum(eigenvalues, mean_eigenvalue)
    
    # Reconstruct the covariance matrix with shrunk eigenvalues
    rie_cov_matrix = eigenvectors @ np.diag(shrunk_eigenvalues) @ eigenvectors.T
    
    return rie_cov_matrix


# Example usage
if __name__ == "__main__":
    # Generate random data (high-dimensional example: p > n)
    np.random.seed(42)
    n_samples = 50
    n_features = 100
    data = np.random.randn(n_samples, n_features)
    
    # Sample covariance matrix
    sample_cov_matrix = np.cov(data, rowvar=False)
    
    # Apply RIE
    rie_cov = compute_rie(sample_cov_matrix)
    
    # Compare with Ledoit-Wolf shrinkage
    lw = LedoitWolf()
    lw_cov = lw.fit(data).covariance_
    
    print("Sample Covariance Matrix (first 3x3):\n", sample_cov_matrix[:3, :3])
    print("\nRIE Covariance Matrix (first 3x3):\n", rie_cov[:3, :3])
    print("\nLedoit-Wolf Covariance Matrix (first 3x3):\n", lw_cov[:3, :3])

Sample Covariance Matrix (first 3x3):
 [[ 8.77332186e-01  1.32027659e-01 -5.83556692e-04]
 [ 1.32027659e-01  1.04297554e+00 -2.48646417e-02]
 [-5.83556692e-04 -2.48646417e-02  9.65413725e-01]]

RIE Covariance Matrix (first 3x3):
 [[ 1.4777527   0.12109452 -0.00254887]
 [ 0.12109452  1.60384943 -0.06827578]
 [-0.00254887 -0.06827578  1.54928346]]

Ledoit-Wolf Covariance Matrix (first 3x3):
 [[ 9.67052462e-01  7.24149375e-03 -3.20070973e-05]
 [ 7.24149375e-03  9.76137719e-01 -1.36378353e-03]
 [-3.20070973e-05 -1.36378353e-03  9.71883585e-01]]
